## Rotten Tomatoes

### Rotten Review

In [1]:
# importar bibliotecas
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import statsmodels.formula.api as smf

In [3]:
# abrir df
df_rotten = pd.read_csv('C:/Users/afilg/OneDrive/Área de Trabalho/DATA SCIENCE/Aulas - Originais/Projeto_Integrador/Dados/rt_movie_info.csv', encoding = 'latin1', engine = 'python', delimiter = ',')

In [4]:
# remover linhas duplicadas
df_rotten.drop_duplicates(subset ="movie_name", 
                     keep = False, inplace = True)

In [5]:
print("Agora o dataset tem:", df_rotten.shape[0], "filmes")

Agora o dataset tem: 11313 filmes


In [6]:
df_rotten.rename(columns = {'movie_link': 'movie_id'}, inplace = True)

### Rotten Review

In [7]:
review_rt = pd.read_csv('C:/Users/afilg/OneDrive/Área de Trabalho/DATA SCIENCE/Aulas - Originais/Projeto_Integrador/Dados/rt_movie_review.csv')

## Análise de Sentimento

https://hub.packtpub.com/how-to-perform-sentiment-analysis-using-python-tutorial/

In [12]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews

In [13]:
import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\afilg\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


True

In [14]:
def extract_features(word_list):
    return dict([(word, True) for word in word_list])

In [15]:
if __name__=='__main__':
   # Load positive and negative reviews  
    positive_fileids = movie_reviews.fileids('pos')
    negative_fileids = movie_reviews.fileids('neg')

In [16]:
features_positive = [(extract_features(movie_reviews.words(fileids=[f])), 'Positive') 
                     for f in positive_fileids]
features_negative = [(extract_features(movie_reviews.words(fileids=[f])), 'Negative') 
                     for f in negative_fileids]

In [17]:
# Split the data into train and test (80/20)
threshold_factor = 0.8
threshold_positive = int(threshold_factor * len(features_positive))
threshold_negative = int(threshold_factor * len(features_negative))

In [18]:
features_train = features_positive[:threshold_positive] + features_negative[:threshold_negative]
#features_test = features_positive[threshold_positive:] + features_negative[threshold_negative:]   

In [19]:
features_test = features_positive[threshold_positive:] + features_negative[threshold_negative:]

In [20]:
# Train a Naive Bayes classifier
classifier = NaiveBayesClassifier.train(features_train)
nltk.classify.util.accuracy(classifier, features_test)

0.735

In [21]:
df_review = review_rt.dropna(subset = ['review_content'])
input_reviews = review_rt.review_content

In [22]:
df_review.shape

(431962, 8)

In [23]:
print("\nPredictions:")
for review in input_reviews:
        print("\nReview:", review)
        probdist = classifier.prob_classify(extract_features(review.split()))
        pred_sentiment = probdist.max()
        print("Predicted sentiment:", pred_sentiment)
        print("Probability:", round(probdist.prob(pred_sentiment), 2))


Predictions:

Review: Has more belly laughs than 10 studio-produced, star-vehicle comedies.
Predicted sentiment: Negative
Probability: 0.73

Review: Calling this story about a quirky high-school student from a dysfunctional family who lives in a goofy small town a one-joke movie is exaggerating by a factor of 10.
Predicted sentiment: Positive
Probability: 0.94

Review: There are lots of laughs for those who enjoy the sight of bottom dwellers doing stupid things that make them look even more idiotic.
Predicted sentiment: Negative
Probability: 0.71

Review: For those who think there'e nothing funnier than yokels acting like idiots for 90 minutes straight.
Predicted sentiment: Negative
Probability: 0.98

Review: We laugh at all the nerdy behavior in the movie, and there's a reason that's OK: Because, deep down, we know we are all nerds.
Predicted sentiment: Negative
Probability: 0.75

Review: He makes Geekus Hollywoodium -- always played by someone such as Ben Stiller, whom we know to be

AttributeError: 'float' object has no attribute 'split'

In [24]:
print("Predicted sentiment:"), pred_sentiment 
print("Probability:"), round(probdist.prob(pred_sentiment), 2)

Predicted sentiment:
Probability:


(None, 0.83)

In [25]:
pred_sentiment = [] 

for review in df_review.review_content:
        probdist = classifier.prob_classify(extract_features(review.split()))
        pred_sentiment.append(probdist.max())
       
df_review['pred_sentiment'] = pred_sentiment  

C:\Users\afilg\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [26]:
df_review.pred_sentiment.value_counts()

Positive    311304
Negative    120658
Name: pred_sentiment, dtype: int64

In [27]:
df_review.head()

,movie_link,movie_id,review_date,critic_publication,critic_icon,critic_top,critic_score,review_content,pred_sentiment
0,https://www.rottentomatoes.com/m/napoleon_dyna...,/m/10007,01/07/2004,Arizona Republic,fresh,Top Critic,4/5,"Has more belly laughs than 10 studio-produced,...",Negative
1,https://www.rottentomatoes.com/m/napoleon_dyna...,/m/10007,01/07/2004,Minneapolis Star Tribune,rotten,Top Critic,0.5/4,Calling this story about a quirky high-school ...,Positive
2,https://www.rottentomatoes.com/m/napoleon_dyna...,/m/10007,02/06/2004,Variety,rotten,Top Critic,NaN,There are lots of laughs for those who enjoy t...,Negative
3,https://www.rottentomatoes.com/m/napoleon_dyna...,/m/10007,02/06/2004,Slant Magazine,rotten,NaN,.5/4,For those who think there'e nothing funnier th...,Negative
4,https://www.rottentomatoes.com/m/napoleon_dyna...,/m/10007,02/07/2004,St. Paul Pioneer Press,fresh,NaN,3.5/4,We laugh at all the nerdy behavior in the movi...,Negative


In [28]:
df_group = pd.pivot_table(df_review, 
                      values='movie_link', 
                      index='movie_id', 
                      columns='pred_sentiment',
                      aggfunc='count',
                      margins = 'True',
                      fill_value = 0)

In [29]:
df_group.head()

pred_sentiment,Negative,Positive,All
movie_id,,,
/m/10007,55,115,170
/m/10010,17,24,41
/m/10014,49,207,256
/m/10015,56,120,176
/m/10016,74,98,172


In [30]:
#calcular sentimento %
df_group['negative_perc'] = df_group['Negative'] / df_group['All'] * 100
df_group['positive_perc'] = df_group['Positive'] / df_group['All'] * 100

In [31]:
df_group.head()

pred_sentiment,Negative,Positive,All,negative_perc,positive_perc
movie_id,,,,,
/m/10007,55,115,170,32.352941,67.647059
/m/10010,17,24,41,41.463415,58.536585
/m/10014,49,207,256,19.140625,80.859375
/m/10015,56,120,176,31.818182,68.181818
/m/10016,74,98,172,43.023256,56.976744


In [36]:
df_group = df_group.reset_index()

In [37]:
df_group.head()

pred_sentiment,movie_id,Negative,Positive,All,negative_perc,positive_perc
0,/m/10007,55,115,170,32.352941,67.647059
1,/m/10010,17,24,41,41.463415,58.536585
2,/m/10014,49,207,256,19.140625,80.859375
3,/m/10015,56,120,176,31.818182,68.181818
4,/m/10016,74,98,172,43.023256,56.976744


In [32]:
## right join os dois datasets do rotten tomato
rotten_df = df_group.merge(df_rotten, on = 'movie_id', how = 'right')

In [33]:
rotten_df.shape

(11313, 36)

In [34]:
rotten_df.head()

,movie_id,Negative,Positive,All,negative_perc,positive_perc,movie_name,rating,genre,directors,...,audience_count,audience_top_critics_count,audience_fresh_critics_count,audience_rotten_critics_count,audience_fresh_top_critics_count,audience_rotten_rotten_critics_count,tomatometer_avg_rating,audience_top_critics_avg_rating,score_sentiment,audience_avg_rating
0,/m/10007,55.0,115.0,170.0,32.352941,67.647059,Napoleon Dynamite,PG,Comedy,Jared Hess,...,789763.0,41,123,50,28,13,6.31,5.92,POSITIVE,3.74
1,/m/10015,56.0,120.0,176.0,31.818182,68.181818,The Notebook,PG-13,"Drama,Romance",Nick Cassavetes,...,31656590.0,44,94,84,19,25,5.64,5.16,POSITIVE,3.61
2,/m/10016,74.0,98.0,172.0,43.023256,56.976744,National Treasure,PG,Action & Adventure,Jon Turteltaub,...,924064.0,43,82,97,16,27,5.30,5.04,POSITIVE,3.85
3,/m/10018,62.0,69.0,131.0,47.328244,52.671756,Christmas With the Kranks,PG,"Comedy,Kids & Family",Joe Roth,...,95709.0,35,7,125,1,34,2.81,3.35,NEGATIVE,2.80
4,/m/10020,46.0,163.0,209.0,22.009569,77.990431,Pirates of the Caribbean: The Curse of the Bla...,PG-13,"Action & Adventure,Science Fiction & Fantasy",Gore Verbinski,...,33125722.0,45,173,46,31,14,7.11,6.44,POSITIVE,3.89


In [38]:
df_group.to_csv('critica.csv', index = False)